In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import os
import numpy as np
import pandas as pd
import string
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA

In [12]:
model_emb = SentenceTransformer('sentence-transformers/LaBSE')
def preprocesar_y_obtener_embedding(texto, pca_cargado, modelo):
    """
    Preprocesa el texto (elimina puntuación y lo convierte a minúsculas),
    obtiene el embedding usando sentence-transformers/LaBSE y lo reduce
    con el PCA cargado.

    Args:
        texto (str): La cadena de texto a procesar.
        pca_cargado (PCA): El modelo PCA previamente cargado.

    Returns:
        numpy.ndarray: El embedding reducido con PCA.
    """

    # Paso 1: Preprocesar el texto
    # Eliminar signos de puntuación y convertir a minúsculas
    texto_limpio = texto.translate(str.maketrans('', '', string.punctuation)).lower()

    # Paso 3: Obtener el embedding del texto
    embedding = modelo.encode([texto_limpio])  # El modelo devuelve una lista de embeddings

    # Paso 4: Reducir la dimensionalidad con el PCA cargado
    embedding_reducido = pca_cargado.transform(embedding)

    return embedding_reducido

In [13]:
data = pd.read_csv('/content/drive/MyDrive/DATATON/dataset_numerico.csv')
data = data.drop(columns="remuneracionAnualCargoPublico")
print(f"Dimensiones del dataset: {data.shape}")
data.head()

Dimensiones del dataset: (74363, 429)


,clave_nivel,cuentaConOtroCargoPublico,clave_ambitoSector,carrera_1,carrera_2,carrera_3,carrera_4,carrera_5,carrera_6,carrera_7,...,puesto_92,puesto_93,puesto_94,puesto_95,puesto_96,puesto_97,puesto_98,puesto_99,puesto_100,remuneracionAnual
0,6,0,0,-0.159850,-0.154407,0.009648,-0.182252,0.318690,-0.179111,-0.024589,...,0.007336,-0.041964,-0.029445,-0.000866,0.015702,0.004752,0.040316,-0.009808,-0.012456,0.002577
1,4,0,1,0.331718,-0.008659,-0.109471,-0.058522,0.105608,-0.050122,0.122084,...,0.006504,-0.024771,-0.001464,0.050382,-0.018026,0.002881,0.002855,-0.012274,0.000190,0.000189
2,4,0,2,-0.338840,0.091707,-0.006823,0.323700,0.344037,-0.337662,0.308657,...,0.004290,-0.025246,-0.018756,0.019450,0.016152,-0.019070,0.018520,-0.071435,-0.031998,0.001384
3,4,0,0,-0.131422,-0.384705,-0.216806,0.026984,-0.143515,0.044224,0.264481,...,0.025571,-0.043006,0.094383,-0.032381,0.020864,-0.001883,0.070095,-0.060599,0.034917,0.000576
4,4,0,0,0.276060,-0.187069,-0.211058,-0.266914,0.389420,0.073814,0.013873,...,-0.059283,0.024034,-0.018491,0.013051,-0.013754,0.006528,-0.005829,-0.039280,-0.014497,0.000273


In [14]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [15]:
# División de los datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

In [16]:
# Crear el modelo de árbol de decisión para regresión
model = DecisionTreeRegressor(random_state=42)
# Entrenar el modelo
model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [17]:
# Predecir con los datos de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 0.00011124612978381874


In [19]:
# Cargar el PCA entrenado
with open('/content/drive/MyDrive/DATATON/pca_areas_50.pkl', 'rb') as f:
    pca_areas = pickle.load(f)

with open('/content/drive/MyDrive/DATATON/pca_carreras_50.pkl', 'rb') as f:
    pca_carreras = pickle.load(f)

with open('/content/drive/MyDrive/DATATON/pca_empleos_100.pkl', 'rb') as f:
    pca_empleos = pickle.load(f)

with open('/content/drive/MyDrive/DATATON/pca_puestos_100.pkl', 'rb') as f:
    pca_puestos = pickle.load(f)

with open('/content/drive/MyDrive/DATATON/pca_funciones_125.pkl', 'rb') as f:
    pca_funciones = pickle.load(f)

In [21]:
# Datos de ejemplo
carrera = "ciencias políticas y administración pública"
empleo = "administración gubernamental"
funcion = "establecer y supervisar políticas educativas"
area = "educación"
puesto = "secretario de educación"

# Obtención de los embeddings y su reducción
emb_carrera = preprocesar_y_obtener_embedding(carrera, pca_carreras, model_emb)
emb_empleo = preprocesar_y_obtener_embedding(empleo, pca_empleos, model_emb)
emb_funcion = preprocesar_y_obtener_embedding(funcion, pca_funciones, model_emb)
emb_area = preprocesar_y_obtener_embedding(area, pca_areas, model_emb)
emb_puesto = preprocesar_y_obtener_embedding(puesto, pca_puestos, model_emb)

# Crear el diccionario con los datos
nuevos_datos = {
    "clave_nivel": 5,
    "cuentaConOtroCargoPublico": 0,
    "clave_ambitoSector": 1
}

# Crear un DataFrame con los embeddings, asignando columnas separadas
nuevos_datos_df = pd.DataFrame([nuevos_datos])

# Aplanar los embeddings para agregarlos al DataFrame
emb_carrera_df = pd.DataFrame(emb_carrera, columns=[f'carrera_{i+1}' for i in range(emb_carrera.shape[1])])
emb_empleo_df = pd.DataFrame(emb_empleo, columns=[f'empleo_{i+1}' for i in range(emb_empleo.shape[1])])
emb_funcion_df = pd.DataFrame(emb_funcion, columns=[f'funcionP_{i+1}' for i in range(emb_funcion.shape[1])])
emb_area_df = pd.DataFrame(emb_area, columns=[f'area_{i+1}' for i in range(emb_area.shape[1])])
emb_puesto_df = pd.DataFrame(emb_puesto, columns=[f'puesto_{i+1}' for i in range(emb_puesto.shape[1])])

# Concatenar los DataFrames con los embeddings al DataFrame principal
nuevos_datos_df = pd.concat([nuevos_datos_df, emb_carrera_df, emb_empleo_df, emb_funcion_df, emb_area_df, emb_puesto_df], axis=1)
print(nuevos_datos_df.shape)

prediction = model.predict(nuevos_datos_df)

import joblib
scaler = joblib.load('/content/drive/MyDrive/DATATON/scaler_remuneracion.pkl')
prediccion_original = scaler.inverse_transform(prediction.reshape(-1, 1))
print(f"La predicción de ganancias anuales es: {prediccion_original}")

(1, 428)
La predicción de ganancias anuales es: [[160469.99999998]]
